In [1]:
%pip install numpy
%pip install pandas plotly


import pandas as pd
from datetime import datetime

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/h2o/miniconda3/envs/torch_cuda/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/h2o/miniconda3/envs/torch_cuda/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/Users/h2o/miniconda3/envs/torch_cuda/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start


AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/h2o/miniconda3/envs/torch_cuda/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/h2o/miniconda3/envs/torch_cuda/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/Users/h2o/miniconda3/envs/torch_cuda/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start


AttributeError: _ARRAY_API not found

In [2]:
import pandas as pd

# Load the CSV data assuming '41.csv' is your filename
data = pd.read_csv('temp.csv')

# Step 1: Drop the first row (repeatedly dropping first index), maintaining your instructions
data = data.drop(index=0).reset_index(drop=True)
data = data.drop(index=0).reset_index(drop=True)

# Step 2: Create a list to hold all the rows that will later form the new DataFrame
rows = []

# Step 3: Iterate over each participant's data to create the new rows
for index, row in data.iterrows():
    pid = row['PROLIFIC_PID']
    response_id = row['ResponseId']
    group = row.get('Group')  # Assuming 'Group' is a column in your data for logic

    # Create a row for 'analogy' with initialized empty 'sketchscore'
    analogy_row = {
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'Technique': 'analogy',
        'sketchscore': None
    }

    # Create a row for 'baseline' with initialized empty 'sketchscore'
    baseline_row = {
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'Technique': 'baseline',
        'sketchscore': None
    }

    # Assign 'Order' based on 'Group'
    if group == '1':
        analogy_row['Order'] = 'first'
        baseline_row['Order'] = 'second'
    elif group == '2':
        analogy_row['Order'] = 'second'
        baseline_row['Order'] = 'first'

    # Add the created rows to the list
    rows.append(analogy_row)
    rows.append(baseline_row)

# Step 4: Create a new DataFrame from the list of rows
new_data = pd.DataFrame(rows, columns=['PROLIFIC_PID', 'ResponseId', 'Order', 'Technique', 'Time', 'PerformanceScore', 'DescriptionScore', 'ChartDifficulties', 'VARK', 'sketchscore'])

# Display the new DataFrame
print(new_data.head())

               PROLIFIC_PID         ResponseId   Order Technique  Time  \
0  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  second   analogy   NaN   
1  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5   first  baseline   NaN   
2  62db2644ab0a3a353c0dcb54  R_2HiGIWFCHg1Nmox   first   analogy   NaN   
3  62db2644ab0a3a353c0dcb54  R_2HiGIWFCHg1Nmox  second  baseline   NaN   
4  629f6b8c65fcae219e245284  R_2NDQ5emeod1xphg   first   analogy   NaN   

   PerformanceScore  DescriptionScore  ChartDifficulties  VARK sketchscore  
0               NaN               NaN                NaN   NaN        None  
1               NaN               NaN                NaN   NaN        None  
2               NaN               NaN                NaN   NaN        None  
3               NaN               NaN                NaN   NaN        None  
4               NaN               NaN                NaN   NaN        None  


In [3]:
rows = []

# Step 3: Iterate over each participant's data to create the new rows
for index, row in data.iterrows():
    pid = row['PROLIFIC_PID']
    response_id = row['ResponseId']
    group = row.get('Group')  # Assuming 'Group' is a column in your data for logic

    # Calculate dominant VARK modality
    vark_types = ['V', 'A', 'R', 'K']
    vark_counts = {v: 0 for v in vark_types}
    for j in range(1, 17):
        vark_col = f'VARK {j}'
        if vark_col in row and pd.notna(row[vark_col]):
            vark_values = str(row[vark_col]).split(',')
            for v in vark_values:
                v = v.strip()
                if v in vark_types:
                    vark_counts[v] += 1
    max_count = max(vark_counts.values())
    dominant_varks = [v for v, count in vark_counts.items() if count == max_count]
    dominant_vark = ''
    for v in vark_types:
        if v in dominant_varks:
            dominant_vark = v
            break

    # Create a row for 'analogy'
    analogy_row = {
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'Technique': 'analogy',
        'sketchscore': None,
        'VARK': dominant_vark
    }

    # Create a row for 'baseline'
    baseline_row = {
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'Technique': 'baseline',
        'sketchscore': None,
        'VARK': dominant_vark
    }

    # Assign 'Order' based on 'Group'
    if group == '1':
        analogy_row['Order'] = 'first'
        baseline_row['Order'] = 'second'
    elif group == '2':
        analogy_row['Order'] = 'second'
        baseline_row['Order'] = 'first'

    # Add the created rows to the list
    rows.append(analogy_row)
    rows.append(baseline_row)

# Step 4: Create a new DataFrame from the list of rows
new_data = pd.DataFrame(rows, columns=['PROLIFIC_PID', 'ResponseId', 'Order', 'Technique', 'Time', 'PerformanceScore', 'DescriptionScore', 'ChartDifficulties', 'VARK', 'sketchscore'])

# Display the new DataFrame
print(new_data.head())


               PROLIFIC_PID         ResponseId   Order Technique  Time  \
0  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  second   analogy   NaN   
1  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5   first  baseline   NaN   
2  62db2644ab0a3a353c0dcb54  R_2HiGIWFCHg1Nmox   first   analogy   NaN   
3  62db2644ab0a3a353c0dcb54  R_2HiGIWFCHg1Nmox  second  baseline   NaN   
4  629f6b8c65fcae219e245284  R_2NDQ5emeod1xphg   first   analogy   NaN   

   PerformanceScore  DescriptionScore  ChartDifficulties VARK sketchscore  
0               NaN               NaN                NaN    K        None  
1               NaN               NaN                NaN    K        None  
2               NaN               NaN                NaN    V        None  
3               NaN               NaN                NaN    V        None  
4               NaN               NaN                NaN    A        None  


In [4]:
key_columns = ['PROLIFIC_PID', 'ResponseId', 'ChartType']
timer_columns_suffix = ['First Click', 'Last Click']


rows = []

# Iterate over each participant's original data row
for index, row in data.iterrows():
    # Extract base information for each entry
    base_info = {k: row[k] for k in key_columns}

    # Collect all timer data by iterating column names
    set_counter = 0
    for timer_index in range(1, 9):  # Timer1 to Timer8
        for click_type in timer_columns_suffix:
            # Pattern to search for multiple occurrences
            timer_pattern = f'Timer{timer_index}_{click_type}'
            occurrences = (col for col in data.columns if timer_pattern in col)

            # For each occurrence in columns, fetch the value
            for timer_col in occurrences:
                if pd.notna(row[timer_col]):
                    new_row = base_info.copy()
                    new_row['TimerType'] = timer_pattern
                    new_row['Set'] = set_counter % 2 + 1
                    new_row['Value'] = row[timer_col]
                    rows.append(new_row)
                    set_counter += 1

    # Ensure only two sets are recorded per aspect
    sets_to_ensure = 2 * len(timer_columns_suffix)
    if set_counter / sets_to_ensure > 2:
        print(f"Warning: More than two set occurrences found for participant: {base_info}")

# Create a DataFrame from the list of rows
timed_click_data = pd.DataFrame(rows)

# Display the first few rows of the DataFrame
print(timed_click_data.head())
print("Total rows:", len(timed_click_data))

               PROLIFIC_PID         ResponseId ChartType           TimerType  \
0  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  BarChart  Timer1_First Click   
1  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  BarChart  Timer1_First Click   
2  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  BarChart   Timer1_Last Click   
3  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  BarChart   Timer1_Last Click   
4  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  BarChart  Timer2_First Click   

   Set   Value  
0    1   7.251  
1    2   5.113  
2    1  66.807  
3    2  62.844  
4    1  27.807  
Total rows: 190


In [5]:
print(timed_click_data.head())

               PROLIFIC_PID         ResponseId ChartType           TimerType  \
0  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  BarChart  Timer1_First Click   
1  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  BarChart  Timer1_First Click   
2  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  BarChart   Timer1_Last Click   
3  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  BarChart   Timer1_Last Click   
4  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  BarChart  Timer2_First Click   

   Set   Value  
0    1   7.251  
1    2   5.113  
2    1  66.807  
3    2  62.844  
4    1  27.807  


In [6]:
# Define key columns and timer column patterns
key_columns = ['PROLIFIC_PID', 'ResponseId', 'ChartType']
timer_columns_suffix = ['First Click', 'Last Click']

# List to store transformed row data
rows = []

# Iterate over each participant's data to extract timer values
for index, row in data.iterrows():
    # Extract base information for each entry
    base_info = {k: row[k] for k in key_columns}

    # Counter to differentiate between sets
    set_counter = 0
    
    for timer_index in range(1, 9):  # Timer1 to Timer8
        for click_type in timer_columns_suffix:
            timer_pattern = f'Timer{timer_index}_{click_type}'
            occurrences = (col for col in data.columns if timer_pattern in col)

            # Pull out each occurrence within timer column
            for timer_col in occurrences:
                if pd.notna(row[timer_col]):
                    new_row = base_info.copy()
                    new_row['TimerType'] = timer_pattern
                    visualization_technique = "analogy" if set_counter % 2 == 0 else "baseline"
                    new_row['VisualizationTechnique'] = visualization_technique
                    new_row['Value'] = row[timer_col]
                    rows.append(new_row)
                    set_counter += 1

# Create a DataFrame from the list of rows
timed_click_data = pd.DataFrame(rows)

# Sort the DataFrame by specified columns
timed_click_data.sort_values(
    by=['PROLIFIC_PID', 'ResponseId', 'ChartType', 'VisualizationTechnique', 'TimerType'],
    inplace=True
)

# Display the sorted DataFrame
print(timed_click_data.head())
print("Total rows:", len(timed_click_data))

                 PROLIFIC_PID         ResponseId ChartType  \
160  61267b828ead584bcf092e35  R_2DtOcX1JcVzppo6  Sunburst   
162  61267b828ead584bcf092e35  R_2DtOcX1JcVzppo6  Sunburst   
164  61267b828ead584bcf092e35  R_2DtOcX1JcVzppo6  Sunburst   
166  61267b828ead584bcf092e35  R_2DtOcX1JcVzppo6  Sunburst   
168  61267b828ead584bcf092e35  R_2DtOcX1JcVzppo6  Sunburst   

              TimerType VisualizationTechnique    Value  
160  Timer1_First Click                analogy   25.264  
162   Timer1_Last Click                analogy  238.241  
164  Timer2_First Click                analogy    0.205  
166   Timer2_Last Click                analogy   88.686  
168  Timer3_First Click                analogy   12.476  
Total rows: 190


In [7]:
# Convert the 'Technique' column to match 'VisualizationTechnique'
new_data['VisualizationTechnique'] = new_data['Technique']

# Initialize list for storing calculated times
timing_summations = []

# Calculate time sums from 'timed_click_data'
for (pid, response_id, visualization_technique), group in timed_click_data.groupby(['PROLIFIC_PID', 'ResponseId', 'VisualizationTechnique']):
    total_time = 0
    for timer_index in range(1, 9):
        first_click_col = f'Timer{timer_index}_First Click'
        last_click_col = f'Timer{timer_index}_Last Click'

        # Extract the values for first and last clicks
        first_clicks = group[group['TimerType'] == first_click_col]['Value']
        last_clicks = group[group['TimerType'] == last_click_col]['Value']

        # Calculate time differences
        if not first_clicks.empty and not last_clicks.empty:
            total_time += sum(float(lc) - float(fc) for lc, fc in zip(last_clicks, first_clicks))

    # Append the result along with identifiers
    timing_summations.append({
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'VisualizationTechnique': visualization_technique,  # Match renamed field
        'Time': total_time
    })

# Convert timing_summations to a DataFrame
time_df = pd.DataFrame(timing_summations)

# Merge 'time_df' with 'new_data' based on matchable columns
final_data = pd.merge(new_data, time_df, on=['PROLIFIC_PID', 'ResponseId', 'VisualizationTechnique'], how='left')

# Drop the redundant 'VisualizationTechnique' column if needed
final_data.drop(columns='VisualizationTechnique', inplace=True)

# Display the final merged DataFrame
print(final_data.head())

               PROLIFIC_PID         ResponseId   Order Technique  Time_x  \
0  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  second   analogy     NaN   
1  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5   first  baseline     NaN   
2  62db2644ab0a3a353c0dcb54  R_2HiGIWFCHg1Nmox   first   analogy     NaN   
3  62db2644ab0a3a353c0dcb54  R_2HiGIWFCHg1Nmox  second  baseline     NaN   
4  629f6b8c65fcae219e245284  R_2NDQ5emeod1xphg   first   analogy     NaN   

   PerformanceScore  DescriptionScore  ChartDifficulties VARK sketchscore  \
0               NaN               NaN                NaN    K        None   
1               NaN               NaN                NaN    K        None   
2               NaN               NaN                NaN    V        None   
3               NaN               NaN                NaN    V        None   
4               NaN               NaN                NaN    A        None   

    Time_y  
0  283.729  
1  652.930  
2  159.009  
3  157.998  
4  456.179  


In [8]:
new_data['VisualizationTechnique'] = new_data['Technique']

# Initialize a list to store timing summations
timing_summations = []

# Calculate time sums and ensure 'ChartType' is included
for (pid, response_id, chart_type, technique), group in timed_click_data.groupby(['PROLIFIC_PID', 'ResponseId', 'ChartType', 'VisualizationTechnique']):
    total_time = 0
    for timer_index in range(1, 9):
        first_click_col = f'Timer{timer_index}_First Click'
        last_click_col = f'Timer{timer_index}_Last Click'

        # Extract values and calculate sums
        first_clicks = group[group['TimerType'] == first_click_col]['Value']
        last_clicks = group[group['TimerType'] == last_click_col]['Value']
        if not first_clicks.empty and not last_clicks.empty:
            total_time += sum(float(lc) for lc, fc in zip(last_clicks, first_clicks))

    # Append result with 'ChartType'
    timing_summations.append({
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'ChartType': chart_type,
        'VisualizationTechnique': technique,
        'TotalTime': total_time
    })

# Create a DataFrame from timing_summations
time_df = pd.DataFrame(timing_summations)

# Merge 'new_data' with 'time_df' on the necessary columns
final_data = pd.merge(new_data, time_df, on=['PROLIFIC_PID', 'ResponseId', 'VisualizationTechnique'], how='left')

# Display the final DataFrame with 'ChartType' included
print(final_data.head())

               PROLIFIC_PID         ResponseId   Order Technique  Time  \
0  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5  second   analogy   NaN   
1  6713a8d7b21cb26dbea856bb  R_23mJW9D7hbbGMW5   first  baseline   NaN   
2  62db2644ab0a3a353c0dcb54  R_2HiGIWFCHg1Nmox   first   analogy   NaN   
3  62db2644ab0a3a353c0dcb54  R_2HiGIWFCHg1Nmox  second  baseline   NaN   
4  629f6b8c65fcae219e245284  R_2NDQ5emeod1xphg   first   analogy   NaN   

   PerformanceScore  DescriptionScore  ChartDifficulties VARK sketchscore  \
0               NaN               NaN                NaN    K        None   
1               NaN               NaN                NaN    K        None   
2               NaN               NaN                NaN    V        None   
3               NaN               NaN                NaN    V        None   
4               NaN               NaN                NaN    A        None   

  VisualizationTechnique       ChartType  TotalTime  
0                analogy        BarCha

In [9]:
final_data.drop(columns=['Time'], inplace=True)

# Display the final DataFrame to confirm changes
final_data.to_csv('dataAnalysis.csv', index=False)